In [2]:
import { StructuredOutputParser } from "@langchain/core/output_parsers";

const parser = StructuredOutputParser.fromNamesAndDescriptions({
  answer: "用户问的问题",
  evidence: "你回答用户问题所依据的答案",
  confidence: "问题答案的可信度评分，格式是百分数",
});

console.log(parser.getFormatInstructions());

You must format your output as a JSON value that adheres to a given "JSON Schema" instance.

"JSON Schema" is a declarative language that allows you to annotate and validate JSON documents.

For example, the example "JSON Schema" instance {{"properties": {{"foo": {{"description": "a list of test words", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}}}
would match an object with one required property, "foo". The "type" property specifies "foo" must be an "array", and the "description" property semantically describes it as "a list of test words". The items within "foo" must be strings.
Thus, the object {{"foo": ["bar", "baz"]}} is a well-formatted instance of this example "JSON Schema". The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.

Your output will be parsed and type-checked according to the provided schema instance, so make sure all fields in your output match the schema exactly and there are no trailing commas!

Here is the JS

## 指令分析

让我们逐段分析这个指令：

### 1. 输出类型声明
首先告诉 LLM 输出的类型：

```
You must format your output as a JSON value that adheres to a given "JSON Schema" instance.
```

### 2. Few-shot 示例
然后，使用 **few-shot**（一种 prompt 技巧），也就是用示例告诉 LLM 什么是 JSON Schema，什么情况会被解析成功，什么情况不会被解析成功：

```
For example, the example "JSON Schema" instance {{"properties": {{"foo":
{{"description": "a list of test words", "type": "array", "items": {{"type":
"string"}}}}}}, "required": ["foo"]}}}} would match an object with one required
property, "foo". The "type" property specifies "foo" must be an "array", and the
"description" property semantically describes it as "a list of test words". The items
within "foo" must be strings. Thus, the object {{"foo": ["bar", "baz"]}} is a well-
formatted instance of this example "JSON Schema". The object {{"properties": {{"foo":
["bar", "baz"]}}}} is not well-formatted.
```

### 3. 重要性强调
然后，再次强调类型的重要性，输出必须遵循给定的 JSON Schema 实例，确保所有字段严格匹配 Schema 中的定义，没有额外的属性，也没有遗漏的必需属性。并强调需要注意细节，比如不要在 JSON 对象中添加多余的逗号，这可能会导致解析失败。

这些 prompt 质量非常高，把在该任务中大模型容易出现的问题都进行了强调，可以有效的保证输出的质量。

### 4. 最终提醒
```
Your output will be parsed and type-checked according to the provided schema instance,
so make sure all fields in your output match the schema exactly and there are no
trailing commas!
```

### 5. JSON Schema 实例
最后才是给出，我们指定的 JSON 格式和对应的描述：

```json
{
  "type": "object",
  "properties": {
    "answer": {
      "type": "string",
      "description": "用户问题的答案"
    },
    "evidence": {
      "type": "string",
      "description": "你回答用户问题所依据的答案"
    },
    "confidence": {
      "type": "string",
      "description": "问题答案的可信度评分，格式是百分数"
    }
  },
  "required": ["answer", "evidence", "confidence"],
  "additionalProperties": false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}
```

In [4]:
import { ChatOpenAI } from "@langchain/openai";
import { PromptTemplate } from "@langchain/core/prompts";
import { envFilePath } from "@/share"
import { load } from "dotenv";

const env = await load({ envPath: envFilePath });
const process = { env };

const prompt = PromptTemplate.fromTemplate(
  `尽可能的回答用的问题 \n{instructions} \n{question}`
);

const chatModel = new ChatOpenAI({
    openAIApiKey: process.env.DEEPSEEK_API_KEY,
    configuration: {
        baseURL: "https://api.deepseek.com/v1/",
    },
    modelName: "deepseek-chat"
})

const chain = prompt.pipe(chatModel).pipe(parser)
const res = await chain.invoke({
  question: "蒙娜丽莎的作者是谁？是什么时候绘制的",
  instructions: parser.getFormatInstructions(),
});

console.log(res);




{
  answer: "蒙娜丽莎的作者是列奥纳多·达·芬奇，大约在1503年至1519年间绘制。",
  evidence: "根据艺术史学家和卢浮宫博物馆（该画作现藏地）的权威资料，列奥纳多·达·芬奇是《蒙娜丽莎》的创作者，创作时期通常被认定为16世纪初（约1503-1506年开始，可能持续到1519年达·芬奇去世前）。",
  confidence: "99%"
}
